 # Beam simulation of GlueX detector hits, take 6 (GlueX Phase 2 beamline)
GlueX detector hits take 6 seeks to reproduce the GlueX Phase 2 photon beam conditions, consisting of 250M simulated beam photons in the range [0.0001,12] GeV from a 50um diamond radiator passing through the 5mm collimator in the Hall D alcove. This simulation subtends approximately 20 milliseconds of beam at 350nA electron beam intensity at 12GeV endpoint energy.

This is a sixth simulation of the Hall D beamline for the KLF experiment, this time under quite different conditions. Instead of the CPS photon source and the KPT conversion target, the standard GlueX photon beam configuration is used: diamond radiator at the standard goniometer position, and the standard Hall D alcove configuration used by GlueX with a 5mm diameter collimator, two sweeping magnets, the pair spectrometer and the shielding wall just upstream of the GlueX liquid hydrogen target. The purpose of this simulation is to provide a baseline of hit rates in the GlueX detectors under known detector operating and triggering conditions, particularly in the start counter, the drift chambes, and the tof counter. This should provide a quantitative basis for evaluation of the rates predicted by the simulation of the klong beam in the CPS + KPT configuration. 

In [1]:
import os
import numpy as np
os.chdir("/srv/jupyter/hdgeant4 studies")
#%pip install --user "gluex.hddm_s==2.3.0"
from gluex import hddm_s
from gluex import xrootd_client as xclient
#%pip install --user "gluex.jupyroot>=1.0.10"
from gluex.jupyroot.hddmview import hddmview
import ROOT
%jsroot on

In [2]:
import dask.distributed
import dask
dclient = dask.distributed.Client(n_workers=50, threads_per_worker=1, dashboard_address='0.0.0.0:8790')

/srv/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8790 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44493 instead
  warnings.warn(


In [3]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/KLFbeam-8-2024/"
infile = xrdurl + xrdpath + "GlueXbeam6_{0}.hddm"

In [4]:
infiles = [infile.format(i+1) for i in range(2500)]
hview = hddmview(infiles, hddm_s, "Klong_hits6.root")
hview.enable_dask_cluster(dclient)

## Hit multiplicities and pulse height spectra in the GlueX detectors

In [5]:
def start_hits_hinit():
    h = {}
    h['hsc'] = ROOT.TH1D("hsc", "start counter hits", 40, 0, 40)
    h['hsc'].GetXaxis().SetTitle("start counter")
    h['hsc'].GetYaxis().SetTitle("counts")
    h['hscrun'] = ROOT.TH1D("hscrun", "start counter hits vs run", 1000, 120000, 130000)
    h['hscrun'].GetXaxis().SetTitle("run number")
    h['hscrun'].GetYaxis().SetTitle("start counter hits")
    h['hdEsc'] = ROOT.TH1D("hdEsc", "start counter pulse height", 1000, 0, 10)
    h['hdEsc'].GetXaxis().SetTitle("dE (MeV)")
    h['hdEsc'].GetYaxis().SetTitle("counts")
    h['htsc'] = ROOT.TH1D("htsc", "start counter pulse time", 1000, 0, 100)
    h['htsc'].GetXaxis().SetTitle("pulse time (ns)")
    h['htsc'].GetYaxis().SetTitle("counts")
    hclones = {}
    for hname in h:
        hname2 = hname + "_dEmin"
        hclones[hname2] = h[hname].Clone(hname2)
        hclones[hname2].SetLineColor(ROOT.kOrange + 7)
    h.update(hclones)
    return h

def start_hits_hfill(record, histos, dEmin=0.5e-3):
    for pev in record.getPhysicsEvents():
        run = pev.runNo
    for hit in record.getStcTruthHits():
        histos['hsc'].Fill(hit.getAttribute("sector"))
        histos['hscrun'].Fill(run)
        histos['hdEsc'].Fill(hit.dE * 1e3)
        histos['htsc'].Fill(hit.t)
        if hit.dE > dEmin:
            histos['hsc_dEmin'].Fill(hit.getAttribute("sector"))
            histos['hscrun_dEmin'].Fill(run)
            histos['hdEsc_dEmin'].Fill(hit.dE * 1e3)
            histos['htsc_dEmin'].Fill(hit.t)
    return histos

n = hview.declare_histograms("start counter", start_hits_hinit, start_hits_hfill)

In [6]:
#hview.fill_histograms()
hview.draw([[['hsc', 'hsc_dEmin'], ['hdEsc', 'hdEsc_dEmin']],
            [['htsc', 'htsc_dEmin'], ]], stats='nie')

hsc_norm = {}
draw_option = "nie"
hview.current_canvas.cd(4)
for hname in ('hsc', 'hsc_dEmin'):
    horig = hview.get(hname)
    hname2 = hname + "_norm"
    hsc_norm[hname2] = horig.Clone(hname2)
    horig.Delete()
    hsc_norm[hname2].SetTitle("start counter hits per 64ns")
    hsc_norm[hname2].Scale(64e-9 / 20e-3)
    hsc_norm[hname2].GetYaxis().SetTitle("counts per 64ns")
    hsc_norm[hname2].Draw(draw_option)
    draw_option = "same"
hview.current_canvas.cd(0)
hview.current_canvas.Draw()

In [17]:
def tracker_hits_hinit():
    h = {}
    h['hcdc'] = ROOT.TH2D("hcdc", "cdc hits", 209, 1, 210, 28, 1, 29)
    h['hcdc'].GetXaxis().SetTitle("straw in ring")
    h['hcdc'].GetYaxis().SetTitle("cdc ring")
    h['hcdcrun'] = ROOT.TH1D("hcdcrun", "cdc hits per run", 1000, 120000, 130000)
    h['hcdcrun'].GetXaxis().SetTitle("run number")
    h['hcdcrun'].GetYaxis().SetTitle("counts")
    h['hdEcdc'] = ROOT.TH1D("hdEcdc", "cdc pulse height", 200, 0, 3e4)
    h['hdEcdc'].GetXaxis().SetTitle("pulse height (q)")
    h['hdEcdc'].GetYaxis().SetTitle("counts")
    h['htcdc'] = ROOT.TH1D("htcdc", "cdc straw hit time", 200, 0, 1000)
    h['htcdc'].GetXaxis().SetTitle("pulse time (ns)")
    h['htcdc'].GetYaxis().SetTitle("counts")
    h['hfdc'] = ROOT.TH2D("hfdc", "fdc hits", 99, 1, 100, 24, 1, 25)
    h['hfdc'].GetXaxis().SetTitle("anode wire")
    h['hfdc'].GetYaxis().SetTitle("fdc layer")
    h['hfdcrun'] = ROOT.TH1D("hfdcrun", "fdc hits per run", 91000, 120000, 130000)
    h['hfdcrun'].GetXaxis().SetTitle("run number")
    h['hfdcrun'].GetYaxis().SetTitle("counts")
    h['hdEfdc'] = ROOT.TH1D("hdEfdc", "fdc pulse height", 200, 0, 10)
    h['hdEfdc'].GetXaxis().SetTitle("dE (KeV)")
    h['hdEfdc'].GetYaxis().SetTitle("counts")
    h['htfdc'] = ROOT.TH1D("htfdc", "fdc anode wire hit time", 200, 0, 500)
    h['htfdc'].GetXaxis().SetTitle("pulse time (ns)")
    h['htfdc'].GetYaxis().SetTitle("counts")
    return h

def tracker_hits_hfill(record, histos):
    for pev in record.getPhysicsEvents():
        run = pev.runNo
    for hit in record.getCdcStrawTruthHits():
        histos['hcdc'].Fill(hit.getAttribute("straw"), hit.getAttribute("ring"))
        histos['hdEcdc'].Fill(hit.q)
        histos['htcdc'].Fill(hit.t)
        if hit.getAttribute("ring") == 1:
            histos['hcdcrun'].Fill(run)
    for hit in record.getFdcAnodeTruthHits():
        histos['hfdc'].Fill(hit.getAttribute("wire"), (hit.getAttribute("module") * 3 - 3) + hit.getAttribute("layer"))
        histos['hdEfdc'].Fill(hit.dE * 1e6)
        histos['htfdc'].Fill(hit.t)
        if hit.getAttribute("module") == 1 and hit.getAttribute("layer") == 1:
            histos['hfdcrun'].Fill(run)
    return histos

n = hview.declare_histograms("drift chambers", tracker_hits_hinit, tracker_hits_hfill)

In [18]:
#hview.fill_histograms()
hview.draw([[['hcdc'], ['hdEcdc']],
            [['htcdc'], ],
            [['hfdc'], ['hdEfdc']],
            [['htfdc'], ],
            [['hfdc'],]],
           [[['colz'], ['']],
            [[''], ],
            [['colz'], ['']],
            [[''], ],
            [['colz'],]], stats='nie'
          )

hdc_norm = {}
canvas_pad = 0
for hname in ("hcdc", "hfdc"):
    hnorm = hname + "_norm"
    hdc_norm[hnorm] = hview.get(hname).ProjectionY(hnorm)
    hdc_norm[hnorm].SetTitle(hname[1:] + " hits per 64ns")
    hdc_norm[hnorm].Scale(64e-9 / 20e-3)
    hdc_norm[hnorm].GetYaxis().SetTitle(hname[1:] + " hits per 64ns")
    hdc_norm[hnorm].SetMinimum(0)
    canvas_pad += 4
    hview.current_canvas.cd(canvas_pad)
    hdc_norm[hnorm].Draw('nie')
canvas_pad = 8
for hname in ("hcdc", "hfdc"):
    hnorm2 = hname + "2d_norm"
    hdc_norm[hnorm2] = hview.get(hname)
    hdc_norm[hnorm2].SetTitle(hname[1:] + " hits per 64ns")
    hdc_norm[hnorm2].Scale(64e-9 / 20e-3)
    hdc_norm[hnorm2].SetMinimum(0)
    canvas_pad += 1
    hview.current_canvas.cd(canvas_pad)
    hdc_norm[hnorm2].Draw('colz')
hview.current_canvas.cd(0)
hview.current_canvas.Draw()

In [19]:
def calorimeter_hits_hinit():
    h = {}
    h['hbcal'] = ROOT.TH2D("hbcal", "bcal hits", 97, -48, 49, 10, 1, 11)
    h['hbcal'].GetXaxis().SetTitle("bcal module (upstream < 0, downstream > 0)")
    h['hbcal'].GetYaxis().SetTitle("bcal layer")
    h['hbcalrun'] = ROOT.TH1D("hbcalrun", "bcal hits per run", 1000, 120000, 130000)
    h['hbcalrun'].GetXaxis().SetTitle("run number")
    h['hbcalrun'].GetYaxis().SetTitle("counts")
    h['hdEubcal'] = ROOT.TH1D("hdEubcal", "upstream bcal pulse height", 1000, 0, 1)
    h['hdEubcal'].GetXaxis().SetTitle("pulse height (GeV)")
    h['hdEubcal'].GetYaxis().SetTitle("counts")
    h['hdEdbcal'] = ROOT.TH1D("hdEdbcal", "downstream bcal pulse height", 1000, 0, 1)
    h['hdEdbcal'].GetXaxis().SetTitle("pulse height (GeV)")
    h['hdEdbcal'].GetYaxis().SetTitle("counts")
    h['htubcal'] = ROOT.TH1D("htubcal", "upstream bcal hit time", 200, 0, 1000)
    h['htubcal'].GetXaxis().SetTitle("pulse time (ns)")
    h['htubcal'].GetYaxis().SetTitle("counts")
    h['htdbcal'] = ROOT.TH1D("htdbcal", "downstream bcal hit time", 200, 0, 1000)
    h['htdbcal'].GetXaxis().SetTitle("pulse time (ns)")
    h['htdbcal'].GetYaxis().SetTitle("counts")
    h['hfcal'] = ROOT.TH2D("hfcal", "fcal hits", 60, 1, 61, 60, 1, 61)
    h['hfcal'].GetXaxis().SetTitle("column")
    h['hfcal'].GetYaxis().SetTitle("row")
    h['hfcalrun'] = ROOT.TH1D("hfcalrun", "fcal hits per run", 1000, 120000, 130000)
    h['hfcalrun'].GetXaxis().SetTitle("run number")
    h['hfcalrun'].GetYaxis().SetTitle("counts")
    h['hdEfcal'] = ROOT.TH1D("hdEfcal", "fcal pulse height", 2000, 0, 10)
    h['hdEfcal'].GetXaxis().SetTitle("dE (GeV)")
    h['hdEfcal'].GetYaxis().SetTitle("counts")
    h['htfcal'] = ROOT.TH1D("htfcal", "fcal block hit time", 200, 0, 500)
    h['htfcal'].GetXaxis().SetTitle("pulse time (ns)")
    h['htfcal'].GetYaxis().SetTitle("counts")
    hclones = {}
    for hname in h:
        hname2 = hname + "_dEmin"
        hclones[hname2] = h[hname].Clone(hname2)
        hclones[hname2].SetLineColor(ROOT.kOrange + 7)
    h.update(hclones)
    return h

def calorimeter_hits_hfill(record, histos, Emin_bcal=0.01, Emin_fcal=0.05):
    for pev in record.getPhysicsEvents():
        run = pev.runNo
    for hit in record.getBcalSiPMUpHits():
        histos['hbcal'].Fill(-hit.getAttribute("module"), hit.getAttribute("layer"))
        histos['hbcalrun'].Fill(run)
        histos['hdEubcal'].Fill(hit.E)
        histos['htubcal'].Fill(hit.t)
        if hit.E > Emin_bcal:
            histos['hbcal_dEmin'].Fill(-hit.getAttribute("module"), hit.getAttribute("layer"))
            histos['hbcalrun_dEmin'].Fill(run)
            histos['hdEubcal_dEmin'].Fill(hit.E)
            histos['htubcal_dEmin'].Fill(hit.t)
    for hit in record.getBcalSiPMDownHits():
        histos['hbcal'].Fill(+hit.getAttribute("module"), hit.getAttribute("layer"))
        histos['hdEdbcal'].Fill(hit.E)
        histos['htdbcal'].Fill(hit.t)
        if hit.E > Emin_bcal:
            histos['hbcal_dEmin'].Fill(+hit.getAttribute("module"), hit.getAttribute("layer"))
            histos['hdEdbcal_dEmin'].Fill(hit.E)
            histos['htdbcal_dEmin'].Fill(hit.t)
    for hit in record.getFcalTruthHits():
        histos['hfcal'].Fill(hit.getAttribute("column"), hit.getAttribute("row"))
        histos['hfcalrun'].Fill(run)
        histos['hdEfcal'].Fill(hit.E)
        histos['htfcal'].Fill(hit.t)
        if hit.E > Emin_fcal:
            histos['hfcal_dEmin'].Fill(hit.getAttribute("column"), hit.getAttribute("row"))
            histos['hfcalrun_dEmin'].Fill(run)
            histos['hdEfcal_dEmin'].Fill(hit.E)
            histos['htfcal_dEmin'].Fill(hit.t)
    return histos

n = hview.declare_histograms("calorimeters", calorimeter_hits_hinit, calorimeter_hits_hfill)

In [20]:
#hview.fill_histograms()
hview.draw([[['hbcal'], ['hdEubcal', 'hdEubcal_dEmin']],
            [['htubcal', 'htubcal_dEmin'], ['hdEdbcal', 'hdEdbcal_dEmin']],
            [['htdbcal', 'htdbcal_dEmin'], ['hfcal']],
            [['hdEfcal', 'hdEfcal_dEmin' ],['htfcal', 'htfcal_dEmin']], 
            [[], ]],
           [[['colz'], ['', '']],
            [['', ''], ['', '']],
            [['', ''], ['colz']],
            [['', ''], ['', '']],
            [[''], ]], stats='nie'
          )
hcal_norm = {}
draw_option = "nie"
canvas_panel = 9
for hname in ("hbcal", "hbcal_dEmin", "hfcal", "hfcal_dEmin"):
    horig = hview.get(hname)
    hnorm = hname + "_norm"
    hcal_norm[hnorm] = horig.ProjectionY(hnorm)
    hcal_norm[hnorm].SetTitle(hname[1:] + " hits per 64ns")
    hcal_norm[hnorm].Scale(64e-9 / 20e-3)
    hcal_norm[hnorm].SetMinimum(0)
    hcal_norm[hnorm].SetStats(0)
    if "bcal" in hname:
        hcal_norm[hnorm].GetYaxis().SetTitle("bcal layers hit per 64ns")
    else:
        hcal_norm[hnorm].GetYaxis().SetTitle("fcal blocks hit per 64ns")
    hview.current_canvas.cd(canvas_panel)
    hcal_norm[hnorm].Draw(draw_option)
    if not "same" in draw_option:
        draw_option = "same"
    else:
        draw_option = "nie"
        canvas_panel += 1
hview.current_canvas.cd(0)
hview.current_canvas.Draw()

In [21]:
def tof_hits_hinit():
    h = {}
    h['htof'] = ROOT.TH1D("htof", "tof counter hits", 101, -50, 51)
    h['htof'].GetXaxis().SetTitle("tof counter (layer 1 < 0, layer 2 > 0)")
    h['htof'].GetYaxis().SetTitle("counts")
    h['htofrun'] = ROOT.TH1D("htofrun", "tof hits per run", 1000, 120000, 130000)
    h['htofrun'].GetXaxis().SetTitle("run number")
    h['htofrun'].GetYaxis().SetTitle("counts")
    h['hdEtof'] = ROOT.TH1D("hdEtof", "pulse height", 1000, 0, 10)
    h['hdEtof'].GetXaxis().SetTitle("dE (MeV)")
    h['hdEtof'].GetYaxis().SetTitle("counts")
    h['httof'] = ROOT.TH1D("httof", "tof counter pulse time", 1000, 0, 100)
    h['httof'].GetXaxis().SetTitle("pulse time (ns)")
    h['httof'].GetYaxis().SetTitle("counts")
    hclones = {}
    for hname in h:
        hname2 = hname + "_dEmin"
        hclones[hname2] = h[hname].Clone(hname2)
        hclones[hname2].SetLineColor(ROOT.kOrange + 7)
    h.update(hclones)
    return h

def tof_hits_hfill(record, histos, dEmin=1e-3):
    for pev in record.getPhysicsEvents():
        run = pev.runNo
    for hit in record.getFtofTruthHits():
        if hit.getAttribute("plane") == 1:
            histos['htof'].Fill(-hit.getAttribute("bar"))
            histos['htofrun'].Fill(run)
        else:
            histos['htof'].Fill(+hit.getAttribute("bar"))
        histos['hdEtof'].Fill(hit.dE * 1e3)
        histos['httof'].Fill(hit.t)
        if hit.dE > dEmin:
            if hit.getAttribute("plane") == 1:
                histos['htof_dEmin'].Fill(-hit.getAttribute("bar"))
                histos['htofrun_dEmin'].Fill(run)
            else:
                histos['htof_dEmin'].Fill(+hit.getAttribute("bar"))
            histos['hdEtof_dEmin'].Fill(hit.dE * 1e3)
            histos['httof_dEmin'].Fill(hit.t)
    return histos

n = hview.declare_histograms("time of flight", tof_hits_hinit, tof_hits_hfill)

In [22]:
#hview.fill_histograms()
hview.draw([[['htof', 'htof_dEmin'], ['hdEtof', 'hdEtof_dEmin']],
            [['httof', 'httof_dEmin'], ]], stats='nie')

htof_norm = {}
draw_option = "nie"
hview.current_canvas.cd(4)
for hname in ('htof', 'htof_dEmin'):
    horig = hview.get(hname)
    hname2 = hname + "_norm"
    hsc_norm[hname2] = horig.Clone(hname2)
    horig.Delete()
    hsc_norm[hname2].SetTitle("tof hits per 64ns")
    hsc_norm[hname2].Scale(64e-9 / 20e-3)
    hsc_norm[hname2].GetYaxis().SetTitle("counts per 64ns")
    hsc_norm[hname2].Draw(draw_option)
    draw_option = "same"
hview.current_canvas.cd(0)
hview.current_canvas.Draw()

In [13]:
def start_hitz_hinit():
    h = {}
    h['hscz'] = ROOT.TH1D("hscz", "start counter hit z", 150, 0, 150)
    h['hscz'].GetXaxis().SetTitle("z (cm)")
    h['hscz'].GetYaxis().SetTitle("counts")
    h['hcdcz'] = ROOT.TH1D("hcdcz", "cdc straw hit z", 250, -50, 200)
    h['hcdcz'].GetXaxis().SetTitle("z (cm)")
    h['hcdcz'].GetYaxis().SetTitle("counts")
    hclones = {}
    for hname in h:
        hname2 = hname + "_dEmin"
        hclones[hname2] = h[hname].Clone(hname2)
        hclones[hname2].SetLineColor(ROOT.kOrange + 7)
    h.update(hclones)
    return h

def start_hitz_hfill(record, histos, dEmin=0.5e-3, qmin=0):
    for pt in record.getStcTruthPoints():
        histos['hscz'].Fill(pt.z)
        dEmax = 0
        for tid in pt.getTrackIDs():
            for hit in record.getStcTruthHits():
                if hit.itrack == tid.itrack and hit.dE > dEmax:
                    dEmax = hit.dE
        if dEmax > dEmin:
            histos['hscz_dEmin'].Fill(pt.z)
    for pt in record.getCdcTruthPoints():
        histos['hcdcz'].Fill(pt.z)
        qmax = 0
        for tid in pt.getTrackIDs():
            for hit in record.getCdcStrawTruthHits():
                if hit.itrack == tid.itrack and hit.q > qmax:
                    qmax = hit.q
        if qmax > qmin:
            histos['hcdcz_dEmin'].Fill(pt.z)
    return histos

n = hview.declare_histograms("inner detector z", start_hitz_hinit, start_hitz_hfill)

In [14]:
hview.fill_histograms()
n = hview.draw([[['hscz', 'hscz_dEmin'], ['hcdcz', 'hcdcz_dEmin']]], stats='nie')

fill_histograms read a total of 2500 tree files, 250000000.0 records


In [15]:
hview.list_histograms(True)

hsc;1           start counter hits                                107929   Sun Oct  6 07:25:57 2024
hscrun;1        start counter hits vs run                         107929   Sun Oct  6 07:25:57 2024
hdEsc;1         start counter pulse height                        107929   Sun Oct  6 07:25:57 2024
htsc;1          start counter pulse time                          107929   Sun Oct  6 07:25:57 2024
hsc_dEmin;1     start counter hits                                 55425   Sun Oct  6 07:25:57 2024
hscrun_dEmin;1  start counter hits vs run                          55425   Sun Oct  6 07:25:57 2024
hdEsc_dEmin;1   start counter pulse height                         55425   Sun Oct  6 07:25:57 2024
htsc_dEmin;1    start counter pulse time                           55425   Sun Oct  6 07:25:57 2024
hcdc;1          cdc hits                                          223516   Sun Oct  6 07:25:57 2024
hcdcrun;1       cdc hits per run                                   15763   Sun Oct  6 07:25:57 2024


49

In [16]:
n = hview.draw('fill_histograms_stats', width=1000)